In [ ]:
# Ensure necessary libraries are installed
!pip install flaml pandas openpyxl scikit-learn


In [ ]:
# Import necessary libraries
import pandas as pd
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Load the Excel file
file_path = '/content/drive/MyDrive/Colab Notebooks/2024_07_24 HW faults, corrective action/inc_trouble_ticket_20240724122454.xlsx'
df = pd.read_excel(file_path)

df.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,SLA Status,Ticket ID,Ticket Status,Business Status,SLA Target Time,Process Type,Business Type,Current Phase,Created By,Processor,...,Alarm Name(Create TT_alarm_name),EMS(Create TT),Domain(Create TT),Sub Domain(Create TT_sub_domain),Equipment Vendor(Create TT_vendor_id),Vendor(Create TT),Site ID(Create TT_site_id),Site Name(Create TT_site_name),Fault First Occur Time(Create TT_first_fault_time),Corrective Action(Process TT)
0,sla_violation,TT-20240630-00000220,Canceled,Canceled,2024-07-01 02:04:19,Trouble Ticket,BusinessProcess,NaN,user:Gavinda Lakmal,NaN,...,RTCH_Ass_Cong_Rate,NaN,IH_Access,2G,Ericsson,NaN,NU0127,Pundaluoya_East,NaN,Automatically Recovered
1,sla_violation,TT-20240702-00000302,Canceled,Canceled,2024-07-02 14:57:49,Trouble Ticket,BusinessProcess,NaN,user:Thilina Wanninayaka,NaN,...,NaN,NaN,Access,4G-TDD,Huawei,NaN,CM0587,Thalahena_East,2024-07-02 12:57:49,Access_VIP_Replace_HW_jumper
2,normal,TT-20240709-00000471,Canceled,Canceled,2024-07-09 16:03:15,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,PLMN Service Unavailable,ENM,Transmission,Microwave,Ericsson,NaN,MO0119,Wedikumbura,2024-07-09 12:00:14,Replaced
3,sla_violation,TT-20240708-00000288,Canceled,Canceled,2024-07-08 16:01:46,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,4G_Traffic_Vol,NaN,IH_Access,Mobile_4G_FDD,Huawei,Huawei,CM5313,Parisara_Mw_Lamp,2024-07-08 09:00:00,Automatically Recovered
4,normal,TT-20240627-00000001,Completed,Closed,2024-06-27 00:44:53,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,Service Unavailable,ENM,Access,4G-Mobile,Ericsson,NaN,CM0051,Kosgama,2024-06-26 23:44:52,Mains Power Recovered
5,sla_violation,TT-20240627-00000003,Completed,Closed,2024-06-27 01:57:57,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,Cell Out of Service,U2020ACC,Access,2G,Huawei,NaN,KU0477,Siyambalangamuwa,2024-06-26 23:57:56,Mains Power Recovered
6,sla_violation,TT-20240627-00000004,Completed,Closed,2024-06-27 03:59:05,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,Cell Out of Service,U2020ACC,Access,2G,Huawei,NaN,KU0420,Buduruwakanda,2024-06-26 23:59:04,Mains_Power_Recovered
7,normal,TT-20240627-00000005,Completed,Closed,2024-06-27 02:10:17,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,Cell Out of Service,U2020ACC,Access,2G,Huawei,NaN,GM0053,Weweldeniya,2024-06-27 00:10:17,Mains Power Recovered
8,sla_violation,TT-20240627-00000007,Completed,Closed,2024-06-27 02:05:31,Trouble Ticket,BusinessProcess,NaN,user:Lalinda Bandara,NaN,...,Cell Out of Service,ENM,Power,Other,NaN,NaN,KL5042,Yatiyana_Temple_Lamp,2024-06-26 22:05:30,Mains Power Recovered
9,sla_violation,TT-20240627-00000008,Completed,Closed,2024-06-28 00:00:04,Trouble Ticket,BusinessProcess,NaN,user:System,NaN,...,CLK_NO_TRACE_MODE,NCE_T,Transmission,Microwave,Huawei,NaN,KU0091,Boyawalana,2024-05-23 18:52:30,Reconfigured


In [ ]:
filtered_df = df[
    (df['EMS(Create TT)'] == 'U2020ACC') |
    (df['Equipment Vendor(Create TT_vendor_id)'] == 'Huawei')
]

filtered_df=filtered_df[(filtered_df['Domain(Create TT)'] == 'Access')]

# List of specific alarm names
alarm_names = [
    "Board Hardware Fault",
    "Radio Signaling Link Disconnected",
    "Cell Capability Degraded",
    "RF Unit CPRI Interface Error",
    "RF Unit VSWR Threshold Crossed",
    "RF Unit Maintenance Link Failure",
    "BBU CPRI Interface Error",
    "RF Unit External Power Supply Insufficient",
    "RF Unit TX Channel Gain Out of Range",
    "RF Unit Hardware Fault",
    "RF Unit DC Input Power Failure",
    "Monitoring Device Maintenance Link Failure",
    "Inter-Board Service Link Failure",
    "BBU Fan Stalled",
    "RF Unit Optical Module Fault",
    "Power Supply Insufficient for Multiple RF Units",
    "RF Unit AC Input Power Failure",
    "Cell RX Channel Interference Noise Power Unbalanced",
    "MAC Excessive Frame Error Rate",
    "Transmission Optical Interface Error",
    "BSL Fault",
    "GNSS Antenna Fault",
    "RF Unit RX Channel RTWP/RSSI Too Low",
    "RF Out of Service",
    "RF Unit Clock Problem",
    "Inter-System Communication Failure",
    "RF Unit Baseband Running Error",
    "BBU Board Maintenance Link Failure",
    "GNSS Locked Satellites Insufficient",
    "Board Temperature Unacceptable",
    "RF Unit Temperature Unacceptable",
    "RHUB CPRI Interface Error",
    "Inter-Board CANBUS Communication Failure",
    "Monitoring Device Power Supply Problem",
    "RF Unit Software Program Error",
    "RF Unit Input Power Out of Range",
    "RHUB-pRRU CPRI Interface Error",
    "RHUB Unit AC Input Power Failure",
    "NR DU Cell TRP Unavailable",
    "BBU CPRI Optical Module Fault",
    "Board Input Voltage Out of Range",
    "Board Powered Off",
    "Board Unavailable",
    "BBU CPRI Line Rate Negotiation Abnormal",
    "Transmission Optical Module Fault"
]

# Assuming df is your DataFrame
filtered_df = filtered_df[filtered_df['Alarm Name(Create TT_alarm_name)'].isin(alarm_names)]


# Drop the columns you don't need
df = filtered_df[['Alarm Name(Create TT_alarm_name)', 'EMS(Create TT)','Domain(Create TT)', 'Sub Domain(Create TT_sub_domain)','Equipment Vendor(Create TT_vendor_id)', 'Site ID(Create TT_site_id)', 'Site Name(Create TT_site_name)', 'Corrective Action(Process TT)']]

df.dropna()

print(df.shape)
# Optionally, display the first few rows of the updated DataFrame
df.head(200)


(453, 8)


,Alarm Name(Create TT_alarm_name),EMS(Create TT),Domain(Create TT),Sub Domain(Create TT_sub_domain),Equipment Vendor(Create TT_vendor_id),Site ID(Create TT_site_id),Site Name(Create TT_site_name),Corrective Action(Process TT)
44,Monitoring Device Maintenance Link Failure,U2020ACC,Access,4G-Mobile,Huawei,KU5148,Dunumawa_Lamp,Automatically_Recovered_Li_Ion_Faulty
47,Board Hardware Fault,U2020ACC,Access,4G-Mobile,Huawei,CM0521,Kuruppu_Rd,QOS_L2_Replace_HW_baseband
56,BBU Board Maintenance Link Failure,U2020ACC,Access,4G-Mobile,Huawei,CM0521,Kuruppu_Rd,Automatically Recovered
59,BBU Fan Stalled,U2020ACC,Access,4G-Fixed,Huawei,PU0117,Karativu_Puttalam,Automatically_Recovered_Li_Ion_Faulty
61,BBU Fan Stalled,U2020ACC,Access,2G,Huawei,PU0145,Nawagattegama,Automatically_Recovered_Li_Ion_Faulty
...,...,...,...,...,...,...,...,...
3409,BBU Fan Stalled,U2020ACC,Access,4G-Fixed,Huawei,CM0116,Unity_Plaza,Automatically_Recovered_Li_Ion_Faulty
3412,RF Unit VSWR Threshold Crossed,U2020ACC,Access,4G-Mobile,Huawei,KU0023,Mawathagama,Automatically Recovered
3419,BBU CPRI Interface Error,U2020ACC,Access,4G-Fixed,Huawei,AN0110,Nuwarawewa,Automatically_Recovered_Li_Ion_Faulty
3422,RF Unit DC Input Power Failure,U2020ACC,Access,4G-Fixed,Huawei,CM0198,Hokandara,Automatically_Recovered


In [ ]:


# Initialize label encoders
le_alarm_name = LabelEncoder()
le_ems = LabelEncoder()
le_domain = LabelEncoder()
le_sub_domain = LabelEncoder()
le_site_id = LabelEncoder()
le_site_name = LabelEncoder()
le_vendor = LabelEncoder()
le_corrective_action = LabelEncoder()

# Fit and transform the columns
filtered_df['Alarm Name'] = le_alarm_name.fit_transform(filtered_df['Alarm Name(Create TT_alarm_name)'].astype(str))
filtered_df['EMS'] = le_ems.fit_transform(filtered_df['EMS(Create TT)'].astype(str))
filtered_df['Domain'] = le_domain.fit_transform(filtered_df['Domain(Create TT)'])
filtered_df['Sub Domain'] = le_sub_domain.fit_transform(filtered_df['Sub Domain(Create TT_sub_domain)'])
filtered_df['Vendor'] = le_vendor.fit_transform(filtered_df['Equipment Vendor(Create TT_vendor_id)'])
filtered_df['Site ID'] = le_site_id.fit_transform(filtered_df['Site ID(Create TT_site_id)'])
filtered_df['Site Name'] = le_site_name.fit_transform(filtered_df['Site Name(Create TT_site_name)'])

# Fit the LabelEncoder on the entire Corrective Action column
le_corrective_action.fit(filtered_df['Corrective Action(Process TT)'])
filtered_df['Corrective Action'] = le_corrective_action.transform(filtered_df['Corrective Action(Process TT)'])

# Check the distribution of classes in y
class_counts = y.value_counts()

# Filter out classes that have only one instance
filtered_classes = class_counts[class_counts > 1].index
filtered_df = filtered_df[filtered_df['Corrective Action'].isin(filtered_classes)]



# Define features (X) and target (y)
X = filtered_df[['Alarm Name', 'EMS', 'Domain', 'Sub Domain']]
y = filtered_df['Corrective Action']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)





In [ ]:
print("Unique labels in y_train:", y_train.unique())
print("Unique labels in y_test:", y_test.unique())


Unique labels in y_train: [6 5 4 9 0 3 7 2]
Unique labels in y_test: [6 5 3 4 7 2 9]


In [ ]:
# Initialize an AutoML instance
automl = AutoML()

# Prepare the settings for AutoML
automl_settings = {
       "time_budget": 60,
    "metric": "accuracy",
    "task": "classification",
    "log_file_name": "automl.log",
    "estimator_list": ["lgbm", "xgboost", "rf"],
    "ensemble": True
}

# Train the model
automl.fit(X_train=X_train, y_train=y_train, **automl_settings)

# Evaluate the model
print(f"Best model: {automl.best_estimator}")
print(f"Accuracy on test set: {automl.score(X_test, y_test)}")

[flaml.automl.logger: 07-24 08:38:12] {1680} INFO - task = classification
[flaml.automl.logger: 07-24 08:38:12] {1691} INFO - Evaluation method: cv


INFO:flaml.automl.task.generic_task:class 0 augmented from 3 to 21
INFO:flaml.automl.task.generic_task:class 1 augmented from 2 to 20
INFO:flaml.automl.task.generic_task:class 2 augmented from 4 to 20
INFO:flaml.automl.task.generic_task:class 6 augmented from 2 to 20
INFO:flaml.automl.task.generic_task:class 7 augmented from 8 to 24


[flaml.automl.logger: 07-24 08:38:12] {1789} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 07-24 08:38:12] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'rf']
[flaml.automl.logger: 07-24 08:38:12] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-24 08:38:12] {2345} INFO - Estimated sufficient time budget=515s. Estimated necessary time budget=1s.
[flaml.automl.logger: 07-24 08:38:12] {2392} INFO -  at 0.1s,	estimator lgbm's best error=0.4041,	best estimator lgbm's best error=0.4041
[flaml.automl.logger: 07-24 08:38:12] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-24 08:38:12] {2392} INFO -  at 0.2s,	estimator lgbm's best error=0.4041,	best estimator lgbm's best error=0.4041
[flaml.automl.logger: 07-24 08:38:12] {2219} INFO - iteration 2, current learner lgbm
[flaml.automl.logger: 07-24 08:38:13] {2392} INFO -  at 0.2s,	estimator lgbm's best error=0.2969,	best estimator lgbm's best error=0.2969


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[flaml.automl.logger: 07-24 08:39:14] {2575} INFO - ensemble: StackingClassifier(estimators=[('rf',
                                <flaml.automl.model.RandomForestEstimator object at 0x7c8d1460b850>),
                               ('xgboost',
                                <flaml.automl.model.XGBoostSklearnEstimator object at 0x7c8d1858fd30>),
                               ('lgbm',
                                <flaml.automl.model.LGBMEstimator object at 0x7c8d195f5270>)],
                   n_jobs=1, passthrough=True)
[flaml.automl.logger: 07-24 08:39:14] {1931} INFO - fit succeeded
[flaml.automl.logger: 07-24 08:39:14] {1932} INFO - Time taken to find the best model: 11.482757329940796
Best model: rf
Accuracy on test set: 0.9438202247191011


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#  already trained the model and have the label encoders
# Example data to predict
example_data = pd.DataFrame({
    'Alarm Name(Create TT_alarm_name)': ['RF Unit TX Channel Gain Out of Range'],
    'Domain(Create TT)': ['Access'],
    'Sub Domain(Create TT_sub_domain)': ['4G-Mobile'],
    'EMS(Create TT_ems)': ['U2020ACC'],  # Replace with actual EMS value
    #'Site ID(Create TT_site_id)': ['CM0521'],  # Replace with actual Site ID
    #'Site Name(Create TT_site_name)': ['Kuruppu_Rd']  # Replace with actual Site Name
})

		#QOS_L2_Replace_HW_baseband

# Encode the example data using the same encoders
example_data['Alarm Name'] = le_alarm_name.transform(example_data['Alarm Name(Create TT_alarm_name)'])
example_data['Domain'] = le_domain.transform(example_data['Domain(Create TT)'])
example_data['Sub Domain'] = le_sub_domain.transform(example_data['Sub Domain(Create TT_sub_domain)'])
example_data['EMS'] = le_ems.transform(example_data['EMS(Create TT_ems)'])
#example_data['Site ID'] = le_site_id.transform(example_data['Site ID(Create TT_site_id)'])
#example_data['Site Name'] = le_site_name.transform(example_data['Site Name(Create TT_site_name)'])

# Select the encoded features for prediction
X_example = example_data[['Alarm Name', 'EMS', 'Domain', 'Sub Domain']]

# Predict the Corrective Action for the example data
predictions = automl.predict(X_example)

# Decode the predicted labels back to the original categories
predicted_corrective_actions = le_corrective_action.inverse_transform(predictions)

# Display the predictions
print(f"Predicted Corrective Actions: {predicted_corrective_actions}")


Predicted Corrective Actions: ['Automatically_Recovered_Li_Ion_Faulty']


In [ ]:
# Assuming you have already trained the model and have the label encoders
# Example data to predict
example_data = pd.DataFrame({
    'Alarm Name(Create TT_alarm_name)': ['RF Unit VSWR Threshold Crossed'],
    'Domain(Create TT)': ['Access'],
    'Sub Domain(Create TT_sub_domain)': ['4G-Mobile'],
    'EMS(Create TT_ems)': ['U2020ACC'],  # Replace with actual EMS value
    #'Site ID(Create TT_site_id)': ['CM0521'],  # Replace with actual Site ID
    #'Site Name(Create TT_site_name)': ['Kuruppu_Rd']  # Replace with actual Site Name
})

		#QOS_L2_Replace_HW_baseband

# Encode the example data using the same encoders
example_data['Alarm Name'] = le_alarm_name.transform(example_data['Alarm Name(Create TT_alarm_name)'])
example_data['Domain'] = le_domain.transform(example_data['Domain(Create TT)'])
example_data['Sub Domain'] = le_sub_domain.transform(example_data['Sub Domain(Create TT_sub_domain)'])
example_data['EMS'] = le_ems.transform(example_data['EMS(Create TT_ems)'])
#example_data['Site ID'] = le_site_id.transform(example_data['Site ID(Create TT_site_id)'])
#example_data['Site Name'] = le_site_name.transform(example_data['Site Name(Create TT_site_name)'])

# Select the encoded features for prediction
X_example = example_data[['Alarm Name', 'EMS', 'Domain', 'Sub Domain']]

# Predict the Corrective Action for the example data
predictions = automl.predict(X_example)

# Decode the predicted labels back to the original categories
predicted_corrective_actions = le_corrective_action.inverse_transform(predictions)

# Display the predictions
print(f"Predicted Corrective Actions: {predicted_corrective_actions}")


Predicted Corrective Actions: ['Automatically Recovered']
